In [16]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2
nest_asyncio.apply()

df1_path = '../dataset/raw/nist/azure_cleaned.csv'
df2_path = '../dataset/raw/nist/aws_cleaned.csv'

output_path = '../dataset/raw/nist/donut-tt-gpt_ver1.csv'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

## FR:

In [18]:
df1 = pd.read_csv(df1_path)
df1.fillna('None', inplace=True)
print(df1.shape)
df1.head()

(2100, 9)


,image_name,DATE,DATE_CONFIDENCE,CITY,CITY_CONFIDENCE,STATE,STATE_CONFIDENCE,ZIP,ZIP_CONFIDENCE
0,f0194_30.png,08/08/89,0.991,"HUNTINGTON BON,",0.287,CA,0.172,92648,None
1,f1421_27.png,08/28/89,0.093,None,None,None,None,None,None
2,f1954_31.png,9-26-89,0.759,RHINELANDER,0.862,Wisconsin,0.976,54501,None
3,f0891_44.png,None,None,Aug 23 1989\nCentral Falls,0.838,KI,0.325,02863,None
4,f1574_26.png,09/16/89,0.823,ALAMOGORDO,0.139,NM,0.86,88350,None


## TT result:

In [19]:
df2 = pd.read_csv(df2_path)
df2.fillna('None', inplace=True)
print(df2.shape)
df2.head()

(2100, 9)


,image_name,DATE,DATE__CONFIDENCE,CITY,CITY__CONFIDENCE,STATE,STATE_CONFIDENCE,ZIP,ZIP_CONFIDENCE
0,f0000_14.png,08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,f0001_41.png,8-7-89,87.28508,Allendale,89.946899,MI,82.12265,49401,77.600266
2,f0002_01.png,8-3-89,81.038841,MINDEN CITY,80.873611,mi,35.721474,48456,37.457157
3,f0003_42.png,08-03-89,87.377403,Charlotte,80.985222,MI,77.612007,48813,71.598038
4,f0004_09.png,08-03-89,89.594048,Holland,80.345238,Mi,62.762756,49424,58.702671


# Load ground truth:

In [2]:
huggingface_dataset_name = "ift/handwriting_forms"

dataset = load_dataset(huggingface_dataset_name)

dataset

Found cached dataset parquet (/home/sohrab/.cache/huggingface/datasets/ift___parquet/ift--handwriting_forms-008fc255b310cae3/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'ground_truth'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['image', 'ground_truth'],
        num_rows: 199
    })
    test: Dataset({
        features: ['image', 'ground_truth'],
        num_rows: 500
    })
})

## Load model

In [3]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, 
                                                       device_map="auto",
                                                       torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [25]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"""trainable model parameters: {trainable_model_params}
                \nall model parameters: {all_model_params}
                \npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"""

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
                
all model parameters: 247577856
                
percentage of trainable model parameters: 100.00%


In [10]:
def tokenize_function(example):
    
    start_prompt = f"""Use the following examples of JSONs to 
    convert the two input JSONs delimited by triple backticks into one output JSON.

    example1: 
    
    JSON1 = {{'DATE': '8-10-89', 'CITY': None, 'STATE': 'Sacramento Ca', 'ZIP': '95841'}}
    JSON2 = {{'DATE': '8-10-89', 'CITY': 'Sacrament', 'STATE': 'None', 'ZIP': 'Ca95841'}}

    OUTPUT_JSON = {{'DATE': 'Sacramento', 'CITY': 'Sacramento', 'STATE': 'Ca', 'ZIP': '95841'}}
    
    
    YOU MUST FOLLOW THE NEXT 8 RULS:
    
    0. DO NOT CHANGE LETTER CASING OF THE INPUT.
    
    1. If the STATE field had a sequence of numbers, they must be moved to the ZIP field. 
    
    2. If the STATE field appears as the full version of the state name and not the abbreviation, keep the full version.     
    
    3. If the STATE field letters apears as capitalised, DO NOT convert the letters to small letters. 
    
    4. If the CITY field letters are capitalised, DO NOT convert the letters to small letters and They MUST kepet as capitalised.
    
    5. DO NOT convert the letters from upper case to lower case. DO NOT change lower case to upper case. 
    
    6. Check the validity of values in the STATE and CITY fields. For example, if the value for the CITY was "ROMEOUILLE" and the value for the STATE field was "ILL",  then the correct spelling for the  CITY field should be "ROMEOVILLE". In such cases, correct the spelling of the CITY field. 

    7. Check the validity of values in the STATE and CITY fields. For example, if the value for the CITY was "New York City" and the STATE field was "NU" filed the spelling for the STATE field and update it to "NY".
    
    OUTPUT_JSON:
    Format your response as a JSON object with all four `keys'.
    OUTPUT_JSON reply MUST be a single json with the following KEYS:

    - DATE
    - CITY
    - STATE
    - ZIP
     text: ```JSON1 = {json1}, JSON2 = {json2}```
    """
    
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

KeyError: 'dialogue'

In [9]:
tokenized_datasets

NameError: name 'tokenized_datasets' is not defined

In [8]:
dataset['train'][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=1 size=2460x350>,
 'ground_truth': '{"gt_parse": {"DATE": "8-3-89", "CITY": "MINDEN CITY", "STATE": "Mi", "ZIP": "48456"}}'}

In [10]:
output_dir = f'./t5-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    do_eval= True,
    learning_rate=1e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_steps=1,
#     max_steps=200
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [11]:
trainer.train()

/home/sohrab/anaconda3/envs/ocr37/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12460
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 31160
  Number of trainable parameters = 247577856


Step,Training Loss
1,29.625000
2,30.500000
3,29.625000
4,30.125000
5,29.125000
6,31.000000
7,31.000000
8,30.625000
9,29.625000
10,29.500000


KeyboardInterrupt: 